In [13]:
import configparser
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, desc

## Creating spark session
It's using a local kubernate cluster to run the spark workers as configured in sparkconf.cfg file.

In [2]:
sparkConf = SparkConf()
parser = configparser.ConfigParser()
parser.optionxform=str
parser.read_file(open('../sparkconf.cfg'))

for section, config in parser.items():
    for key, value in config.items():
        sparkConf.set(key, value)

spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()
spark

## Loading tables
Loading the fact and dimension tables from s3 buckets

In [3]:
input_data = "s3a://dutrajardim/udacity-dl-project/"

df_songplays = spark.read.format('parquet').load('%ssongplays.parquet' % input_data).alias("songplays")
df_times = spark.read.format('parquet').load('%stimes.parquet' % input_data).alias("times")
df_artists = spark.read.format('parquet').load('%sartists.parquet' % input_data).alias("artists")
df_users = spark.read.format('parquet').load('%susers.parquet' % input_data).alias("users")
df_songs = spark.read.format('parquet').load('%ssongs.parquet' % input_data).alias("songs")

22/01/01 06:20:42 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
22/01/01 06:20:43 WARN AmazonHttpClient: SSL Certificate checking for endpoints has been explicitly disabled.


### Creating a olap cube with artist name, user level and weekday

In [8]:
# artists.name, songplays.level, times.weekday
cube_level_weekday_name = \
    df_songplays \
        .join(df_times, on='start_time', how='left') \
        .join(df_artists, on='artist_id', how='left') \
        .fillna("unknown", subset=['name']) \
        .cube('level', 'weekday', 'name') \
        .count()

#### Counting song plays plays by artist (top 10)

The *unknown* counting is related to song plays in the log dataset for which there wasn't match in the song dataset.

In [18]:
cube_level_weekday_name \
    .where(expr("level IS NULL AND weekday IS NULL AND name IS NOT NULL")) \
    .select('name', 'count') \
    .sort(desc('count')) \
    .limit(10) \
    .show()

+--------------------+-----+
|                name|count|
+--------------------+-----+
|             unknown| 7734|
|       Dwight Yoakam|   37|
|Kid Cudi / Kanye ...|   10|
|            Kid Cudi|   10|
|       Lonnie Gordon|    9|
|          Ron Carter|    9|
|               B.o.B|    8|
|                Muse|    6|
|               Usher|    6|
|Usher featuring J...|    6|
+--------------------+-----+



#### Counting song plays plays by weekday

In [37]:
cube_level_weekday_name \
    .where(expr("level IS NULL AND weekday IS NOT NULL AND name IS NULL")) \
    .select('weekday', 'count') \
    .sort('weekday') \
    .show()

+-------+-----+
|weekday|count|
+-------+-----+
|      1|  504|
|      2| 1228|
|      3| 1268|
|      4| 1619|
|      5| 1244|
|      6| 1527|
|      7|  716|
+-------+-----+



#### Counting song plays by user level

In [36]:
cube_level_weekday_name \
    .where(expr("level IS NOT NULL AND weekday IS NULL AND name IS NULL")) \
    .select('level', 'count') \
    .sort(desc('count')) \
    .show()

+-----+-----+
|level|count|
+-----+-----+
| paid| 6324|
| free| 1782|
+-----+-----+



### Creating a olap cube with user gender, song year and week of the year

In [30]:

cube_year_week_gender = \
    df_songplays.select('start_time', 'user_id', 'song_id') \
        .join(df_times.select('start_time', 'week'), on='start_time', how='left') \
        .join(df_users.select('user_id', 'gender'), on='user_id', how='left') \
        .join(df_songs.select('song_id', 'year'), on='song_id', how='left') \
        .fillna({
            'gender': "unknown",
            'year': -1
        }) \
        .cube('year', 'week', 'gender') \
        .count()

#### Counting song plays by gender

The *unknown* counting is related to anonymous users.

In [35]:
cube_year_week_gender \
    .where(expr("year IS NULL AND week IS NULL AND gender IS NOT NULL")) \
    .select('gender', 'count') \
    .sort(desc('count')) \
    .show()

+-------+-----+
| gender|count|
+-------+-----+
|      F| 5482|
|      M| 2288|
|unknown|  286|
+-------+-----+



#### Counting song plays from female users by week 

In [34]:
cube_year_week_gender \
    .where(expr("year IS NULL AND week IS NOT NULL AND gender = 'F'")) \
    .select('week', 'count') \
    .sort('week') \
    .show()

+----+-----+
|week|count|
+----+-----+
|  44|  322|
|  45| 1004|
|  46| 1695|
|  47| 1397|
|  48| 1064|
+----+-----+



#### Counting song plays from male users by song year

The result shows the count for the songs of the last 10 years in the dataset. The count is only related to songs data in log dataset that is also in the song dataset.

In [49]:
cube_year_week_gender \
    .where(expr("year > 0 AND week IS NULL AND gender = 'M'")) \
    .select('year', 'count') \
    .sort(desc('year')) \
    .limit(10) \
    .show()

+----+-----+
|year|count|
+----+-----+
|2010|    6|
|2009|    7|
|2008|    4|
|2007|    7|
|2006|    4|
|2005|    1|
|2004|    4|
|2003|    5|
|2002|    3|
|2001|    2|
+----+-----+



## Closing spark session

In [50]:
spark.stop()

22/01/01 07:57:10 WARN ExecutorPodsWatchSnapshotSource: Kubernetes client has been closed.
